# Exemple de finetuning d'un modèle BERT sur la classification des avis du site Allociné

[Ouvrir dans Google colab](https://colab.research.google.com/github/Herve07h22/nlp-fine-tuning/blob/main/allocine.ipynb)

N'oubliez pas de sléectionner "GPU" dans l'environnement d'exécution du notebook, car ce n'est pas activé par défaut.

In [ ]:
%pip install datasets transformers

In [4]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import torch

In [ ]:
raw_datasets = load_dataset("allocine")
raw_datasets.shape

## aperçu du dataset "Allociné"

5 commentaires positifs

In [ ]:
positive_samples = raw_datasets["train"].shuffle(seed=42).filter(lambda example: example['label']==1)
positive_samples['review'][:5]

5 commentaires négatifs

In [ ]:
positive_samples = raw_datasets["train"].shuffle(seed=42).filter(lambda example: example['label']==0)
positive_samples['review'][:5]

## Pre-processing du dataset

Pour l'entraînement du modèle de classification, on utiliser un `Trainer` de HuggingFace. Il utilise un dataset comportant 2 features :
- le label (0 ou 1)
- les input_ids tokenisés
- les attention_mask (1 si le mot est présent, 0 sinon)

On va donc pré-processer notre dataset avec le Tokeniser du modèle

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("camembert-base")

def tokenize_function(examples):
    return tokenizer(examples["review"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

tokenized_datasets.shape

On sélectionne un sous-ensemble de données pour entraîner le modèle.

L'objectif est d'évaluer comment la précision augmente avec le volume de données.

In [9]:
TRAIN_DATASET_SIZE = 100
TEST_DATASET_SIZE = 1000
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(TRAIN_DATASET_SIZE))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(TEST_DATASET_SIZE))

On utilise le modèle BERT [camembert-base](https://camembert-model.fr).



In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("camembert-base", num_labels=2)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

In [11]:
training_args = TrainingArguments("test_trainer")

On utilise la fonction de calcul de métrique disponible dans le package `datasets`.

On s'intéresse à la précision = (vrais positifs + vrais négatifs) / (taille du dataset de validation)


In [12]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


La classe `Trainer` est une abstraction de du package `transformers` de HuggingFace permettant de fine-tuner le modèle très simplement.

In [13]:
trainer = Trainer(model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset, compute_metrics=compute_metrics)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

| train data set size | training time (google colab) | accuracy | loss |
| ------------------- | ---------------------------- | -------- | ---- |
| 100 | 26s | 91.8 | 0.489 |
| 200 | 1m46 | 95.2 | 0.194 |
| 300 | 2m40 | 95.6 | 0.169 |
| 500 | 2m19s | 95.5 | 0.2259 |
| 1000 | 4m46s | 95.6 | 0.2269 |